In [0]:
from google.colab import drive
import warnings
import pandas as pd
import numpy as np
drive.mount('/gdrive')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets'
warnings.filterwarnings('ignore')
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets'
fasttext_data = pd.read_csv('2020_05_09_encoded_Fasttext.csv', index_col=0)
print(fasttext_data.shape)
base_data = pd.read_csv('papildinatie_jautajumi_cleaned.csv', index_col=0)
print(base_data.shape)
%cd '/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded'
bert_big_data = pd.read_csv('2020_05_10_encoded_BERT_big.csv', index_col=0)
bert_multi_cased_data = pd.read_csv('2020_05_10_encoded_BERT_multi_cased.csv', index_col=0)
bert_small_2_data = pd.read_csv('2020_05_10_encoded_BERT_small_2.csv', index_col=0)
bert_small_data = pd.read_csv('2020_05_10_encoded_BERT_small.csv', index_col=0)
print(bert_big_data.shape, bert_multi_cased_data.shape, bert_small_2_data.shape, bert_small_data.shape)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets
/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets
(23391, 301)
(23391, 2)
/gdrive/My Drive/LU/LU DF/8. semestris/Bakalaura Darbs/Datasets/BERT_encoded
(23391, 769) (23391, 769) (23391, 257) (23391, 257)


In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.preprocessing import text
import matplotlib.pyplot as plt
from keras import layers
from keras import models
import datetime
from imblearn.under_sampling import ClusterCentroids, NearMiss, RandomUnderSampler
from imblearn.base import BaseSampler

Using TensorFlow backend.


In [0]:
class FakeSampler(BaseSampler):
    _sampling_type = 'bypass'
    def _fit_resample(self, X, y):
        return X, y

In [0]:
def transformResults(results):
    mod_results = []
    for result in results:
        out = np.argmax(result)
        mod_results.append(out)
    mod_results = np.array(mod_results)  
    return mod_results 

In [0]:
oversampler  = FakeSampler()
indices       = range(3)
dataset_list = [bert_small_2_data, bert_small_data, bert_multi_cased_data, bert_big_data, fasttext_data]
model_names  = ['BERT small #2', 'BERT small', 'BERT multi cased', 'BERT big', 'FastText']
n_split = 5
counter = 0 
results1 = []
header_row = ['Modelis', 'RandomUnderSampler', 'NearMiss', 'ClusterCentroids']
results1.append(header_row)

In [0]:
exec_start = datetime.datetime.now()
for dataset in dataset_list:
  result_row = []
  result_row.append(model_names[counter])
  # Extracting the features and the target based on dimensionality of each dataset
  col_count = len(dataset.columns)
  features = dataset.iloc[:,:col_count-1]
  target = dataset.iloc[:,col_count-1:]

  features = features.to_numpy()
  target = target.to_numpy()

  for i, oversampler in zip( indices, (RandomUnderSampler(),
                                       NearMiss(),
                                       ClusterCentroids()
                                        )):
    os_name = oversampler.__class__.__name__
    date_start    = datetime.datetime.now()                                     # Fiksējam Oversampling sākuma laiku     
    X_res, y_res = oversampler.fit_resample(features, target)                   # Veicam Oversampling
    date_end = datetime.datetime.now()                                          # Fiksējam beigu laiku
    time_elapsed_us = date_end - date_start                                     # Fiksējam Oversampling patērēto laiku
    print('UnderSampled dataset shape:', len(X_res), len(X_res[0]))

    model_f1_scores = []
    fold_counter = 0
    for train_index,test_index in StratifiedKFold(n_split).split(X_res, y_res):
      # creating train and test datasets
      x_train,x_test=X_res[train_index],X_res[test_index]
      y_train,y_test=y_res[train_index],y_res[test_index]  
      # Firstly, creating the neural network model
      model = models.Sequential()
      model.add(layers.Dense(256, activation='relu', input_shape=(col_count-1,)))
      model.add(layers.Dense(64, activation='relu'))
      model.add(layers.Dense(4, activation='softmax'))
      # Then, compiling the created model
      model.compile(loss=['sparse_categorical_crossentropy'],
                    optimizer='adam',
                    metrics=['sparse_categorical_accuracy'])
      batch_size = 64
      epochs = 5
      # Then, fitting the model
      history = model.fit(x_train, y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=0)
      y_pred = model.predict(x_test)                                                  # Prasam modelim paredzēt testa datiem kategorijas
      y_pred2 = transformResults(y_pred)                                              # Pārveidojam rezultātus

      model_f1_avg = round(f1_score(y_test, y_pred2, average='macro'),4)              # Fiksējam f1 vidējo rezultātu kārtējā K-Foldā
      model_f1_scores.append(model_f1_avg)
      fold_counter = fold_counter + 1
      print('\tCompleted for model: ',model_names[counter],' K-Fold #', fold_counter, 'F1 score:', model_f1_avg)
  
    #After the K-Fold validation
    model_f1 = round(np.mean(model_f1_scores), 4)
    result_row.append(model_f1)
    print('Completed for:', os_name,' with F1 score:',model_f1, ' Oversampling duration:',time_elapsed_us)  
  #After all four UnderSamplers
  results1.append(result_row)
  print('Completed for ', model_names[counter], ' with results=', result_row)
  counter = counter + 1

#In the very end - fix the execution time
exec_end = datetime.datetime.now()
exec_elapsed = exec_end - exec_start
print('Total time elapsed:', exec_elapsed)

# Saglabājam visus rezultātus
result_data = pd.DataFrame(results1[1:], columns=results1[0])
result_data

UnderSampled dataset shape: 488 256
	Completed for model:  BERT small #2  K-Fold # 1 F1 score: 0.6419
	Completed for model:  BERT small #2  K-Fold # 2 F1 score: 0.5538
	Completed for model:  BERT small #2  K-Fold # 3 F1 score: 0.7439
	Completed for model:  BERT small #2  K-Fold # 4 F1 score: 0.558
	Completed for model:  BERT small #2  K-Fold # 5 F1 score: 0.6494
Completed for: RandomUnderSampler  with F1 score: 0.6294  Oversampling duration: 0:00:00.011460
UnderSampled dataset shape: 488 256
	Completed for model:  BERT small #2  K-Fold # 1 F1 score: 0.4374
	Completed for model:  BERT small #2  K-Fold # 2 F1 score: 0.5959
	Completed for model:  BERT small #2  K-Fold # 3 F1 score: 0.7781
	Completed for model:  BERT small #2  K-Fold # 4 F1 score: 0.8254
	Completed for model:  BERT small #2  K-Fold # 5 F1 score: 0.741
Completed for: NearMiss  with F1 score: 0.6756  Oversampling duration: 0:00:01.794017
UnderSampled dataset shape: 488 256
	Completed for model:  BERT small #2  K-Fold # 1 F1 

,Modelis,RandomUnderSampler,NearMiss,ClusterCentroids
0,BERT small #2,0.6294,0.6756,0.6468
1,BERT small,0.7798,0.7785,0.8167
2,BERT multi cased,0.7269,0.7574,0.7719
3,BERT big,0.8557,0.8992,0.8959
4,FastText,0.7614,0.8128,0.8105


In [0]:
result_data.to_csv('2020_05_13_papildinatie_visi_UnderSampling_results.csv')

<h2>Bāzlīnija</h2>


In [0]:
from keras.preprocessing import text
from keras import layers
from keras import models
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold 

In [0]:
result_table = []
header_row = ['Vārdnīcas izmērs', 'Oversampler', 'Rezultāti']
result_table.append(header_row)

In [0]:
print(base_data.shape)
base_data.head(3)

(23391, 2)


,Question,labels
0,septembrī noslēgt dv līgumu tas pats darba laiks,2
1,strādās bibliotēka,2
2,kāds darba laiks kab piektdienā lūdzu atbildie...,2


In [0]:
col_count = len(base_data.columns)
features = base_data['Question'].to_numpy().astype('str')
target = base_data['labels']

In [0]:
max_words = 2000
# Text pre-processing
tokenize = text.Tokenizer(num_words=max_words, char_level=False)                # izveidojam Tokenizer ar noteikto vārdnīcas izmēru
tokenize.fit_on_texts(features)                                                 # apmācām Tokenizer uz mūsu datiem
X = tokenize.texts_to_matrix(features)                                          # pārveidojam treniņdatus par tekstvienību vektoriem
print('Tokenizer done:', max_words, X.shape)

Tokenizer done: 2000 (23391, 2000)


<h3>ClusterCentroids nav iespējams darbināt ar Bāzlīniju Naive Bayes, jo tā vērtības ir negatīvas</h3>

In [0]:
exec_start = datetime.datetime.now()
n_split = 5                                                                     # at each fold, we use 80% / 20% split
indices = range(3)
for i, undersampler in zip( indices, (RandomUnderSampler(),
                                       NearMiss(),
                                       ClusterCentroids()
                                        )):
  us_name = undersampler.__class__.__name__
  date_start    = datetime.datetime.now()                                     # Fiksējam Undersampling sākuma laiku     
  X_res, y_res = undersampler.fit_resample(X, target)                         # Veicam UnderSampling
  date_end = datetime.datetime.now()                                          # Fiksējam beigu laiku
  time_elapsed_us = date_end - date_start                                     # Fiksējam Undersampling patērēto laiku
  print('Undersampled dataset shape:', len(X_res), len(X_res[0]))
  result_row = []
  result_row.append(max_words)
  result_row.append(us_name)
 
  f1_scores_naive = []                                                          # Creating an empty array of f1 scores 
  f1_scores_keras = []
  counter = 0
  for train_index,test_index in StratifiedKFold(n_split).split(X_res, y_res):
    # creating train and test datasets
    x_train,x_test=X_res[train_index],X_res[test_index]
    y_train,y_test=y_res[train_index],y_res[test_index]

    ### Naive Bayes ###
    naive_bayes = MultinomialNB()                                               # Izveidojam Naive Bayes modeli
    naive_bayes.fit(x_train, y_train)                                           # Trenējam modeli uz treniņdatiem
    naive_y_pred = naive_bayes.predict(x_test)
    naive_f1_avg = round(f1_score(y_test, naive_y_pred, average='macro'),4)     # Fiksējam Naive Bayes f1 vidējo rezultātu
    f1_scores_naive.append(naive_f1_avg)

    ### Keras Sequential ###
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(max_words,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))

    model.compile(loss=['sparse_categorical_crossentropy'],
                optimizer='adam',
                metrics=['sparse_categorical_crossentropy'])
    batch_size = 64
    epochs = 5
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)
    y_pred = model.predict(x_test)                                              # Prasam modelim paredzēt testa datiem kategorijas
    keras_y_pred = transformResults(y_pred)                                     # Pārveidojam rezultātus, paņemot ticamāko klasi no varbūtībām
    keras_f1_avg = round(f1_score(y_test, keras_y_pred, average='macro'),4)     # Fiksējam Naive Bayes f1 vidējo rezultātu
    f1_scores_keras.append(keras_f1_avg)
    counter = counter + 1
    print('\tCompleted for vocabulary size:', max_words,' K-Fold #',counter, ' Naive Bayes score:',naive_f1_avg, ' Keras score:',keras_f1_avg)
    
  #After the K-Fold validation
  f1_avg_naive = round(np.mean(f1_scores_naive), 4)
  f1_avg_keras = round(np.mean(f1_scores_keras),4)
  result_text = str(f1_avg_naive) + '/' + str(f1_avg_keras)
  print('Completed for vocabulary size:', max_words,' Naive Bayes/Keras score:',result_text)
  result_row.append(result_text)
  result_table.append(result_row)
#In the very end - fix the execution time
exec_end = datetime.datetime.now()
exec_elapsed = exec_end - exec_start
print('Total time elapsed:', exec_elapsed)

# Saglabājam visus rezultātus
result_data = pd.DataFrame(result_table[1:], columns=result_table[0])
result_data.to_csv('2020_05_13_papildinatie_Bāzlīnija_UnderSampling_results.csv')
result_data

Undersampled dataset shape: 488 2000
	Completed for vocabulary size: 2000  K-Fold # 1  Naive Bayes score: 0.8258  Keras score: 0.8546
	Completed for vocabulary size: 2000  K-Fold # 2  Naive Bayes score: 0.8642  Keras score: 0.9167
	Completed for vocabulary size: 2000  K-Fold # 3  Naive Bayes score: 0.8224  Keras score: 0.8603
	Completed for vocabulary size: 2000  K-Fold # 4  Naive Bayes score: 0.812  Keras score: 0.8331
	Completed for vocabulary size: 2000  K-Fold # 5  Naive Bayes score: 0.8248  Keras score: 0.8155
Completed for vocabulary size: 2000  Naive Bayes/Keras score: 0.8298/0.856
Undersampled dataset shape: 488 2000
	Completed for vocabulary size: 2000  K-Fold # 1  Naive Bayes score: 0.9059  Keras score: 0.8836
	Completed for vocabulary size: 2000  K-Fold # 2  Naive Bayes score: 0.9493  Keras score: 0.9493
	Completed for vocabulary size: 2000  K-Fold # 3  Naive Bayes score: 0.6618  Keras score: 0.6571
	Completed for vocabulary size: 2000  K-Fold # 4  Naive Bayes score: 0.5996 

ValueError: ignored